Import necessary libraries

In [10]:
import pandas as pd
import requests
import os
from datetime import datetime, timedelta
import numpy as np
import gc



# Downloading data (Traffic and Air quality). 
Automatically downloads from the following dates :

In [11]:

download_from = datetime(2024, 2, 1)  # Example: January 2023
download_until = datetime(2024, 2, 28) # Or set a specific end date

Then run the following to actually download the files.

In [12]:

# Catalan month names
catalan_months = {
    1: "Gener",
    2: "Febrer",
    3: "Marc",
    4: "Abril",
    5: "Maig",
    6: "Juny",
    7: "Juliol",
    8: "Agost",
    9: "Setembre",
    10: "Octubre",
    11: "Novembre",
    12: "Desembre",
}

# Trams transit relacio (relation between ids and locations)
trams_relacio_url = "https://opendata-ajuntament.barcelona.cat/data/dataset/1090983a-1c40-4609-8620-14ad49aae3ab/resource/1d6c814c-70ef-4147-aa16-a49ddb952f72/download/transit_relacio_trams.csv"
trams_relacio_path = "./data/transit_relacio_trams.csv"

# Air quality stations info (including lat and long)
# (The stations are unchanged since 2023 so downloading only the 2025 version is enough)
air_stations_info_url = "https://opendata-ajuntament.barcelona.cat/data/dataset/4dff88b1-151b-48db-91c2-45007cd5d07a/resource/d1aa40d7-66f9-451b-85f8-955b765fdc2f/download/2025_qualitat_aire_estacions.csv"
air_stations_info_path = "./data/air_stations_info.csv"

def generate_urls(download_from, download_until):
    urls = []
    current_date = datetime(download_from.year, download_from.month, 1)
    end_date = datetime(download_until.year, download_until.month, 1)

    while current_date <= end_date:
        year = current_date.year
        month = current_date.month
        month_name = catalan_months[month]

        # Generate URLs for both datasets
        tram_url = f"https://opendata-ajuntament.barcelona.cat/resources/auto/transit/{year}_{month:02d}_{month_name}_TRAMS_TRAMS.csv"
        aire_url = f"https://opendata-ajuntament.barcelona.cat/resources/bcn/QualitatAire/{year}_{month:02d}_{month_name}_qualitat_aire_BCN.csv"

        tram_filename = f"data/TRAMS_{year}_{month:02d}_{month_name}.csv"
        aire_filename = f"data/QualitatAire_{year}_{month:02d}_{month_name}.csv"

        urls.append((tram_url, tram_filename))
        urls.append((aire_url, aire_filename))

        # Move to the next month
        if current_date.month == 12:
            current_date = datetime(current_date.year + 1, 1, 1)
        else:
            current_date = datetime(current_date.year, current_date.month + 1, 1)

    return urls

def download_file(url, filename):
    if os.path.exists(filename):
        print(f"File already exists: {filename}. Skipping download.")
        return

    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()
        with open(filename, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        print(f"Downloaded: {filename}")
    except Exception as e:
        print(f"Failed to download {filename}: {e}")


# Create a "data" folder:
if not os.path.exists("./data"):
    os.mkdir("./data")

urls = generate_urls(download_from, download_until)

print("Dwonloading files...")
for url, filename in urls:
    download_file(url, filename)


# Then download the TRAMS relacio file
if not os.path.exists(trams_relacio_path):
    download_file(trams_relacio_url, trams_relacio_path)


# And the air stations info 
if not os.path.exists(air_stations_info_path):
    download_file(air_stations_info_url, air_stations_info_path)

Dwonloading files...
File already exists: data/TRAMS_2024_02_Febrer.csv. Skipping download.
File already exists: data/QualitatAire_2024_02_Febrer.csv. Skipping download.


# Data treatment and cleaning


## Creating a merged dataset

### Combining all the air quality files

In [13]:
## Create dataframe of measurements

air_quality_files = [filename for url, filename in urls if "QualitatAire" in filename]
#print(air_quality_files)

# Covnert to pandas dataframes
air_quality_dfs = [pd.read_csv(file) for file in air_quality_files]
# Combine them into a single dataframe
air_quality = pd.concat(air_quality_dfs, ignore_index=True)
#print(air_quality_combined.head())



# Only keep PM10 DATA
air_quality = air_quality[air_quality["CODI_CONTAMINANT"].isin([10, 110])]
print(air_quality.head())




## Create dataframe of station info
air_station_info = pd.read_csv(air_stations_info_path)
# Only keep location data for each station
air_station_locations = air_station_info[['Estacio', 'Latitud', 'Longitud']].drop_duplicates(subset=['Estacio'])
#print(air_station_locations.head(n=20))

""" ## Now merge the air_quality measurements and the location

air_quality = air_quality_combined.merge(
    right=air_station_locations,
    left_on='ESTACIO',
    right_on='Estacio',
    how='left'
).drop(columns=['Estacio'])

print(air_quality.head()) """

    CODI_PROVINCIA  PROVINCIA  CODI_MUNICIPI   MUNICIPI  ESTACIO  \
58               8  Barcelona             19  Barcelona        4   
59               8  Barcelona             19  Barcelona        4   
60               8  Barcelona             19  Barcelona        4   
61               8  Barcelona             19  Barcelona        4   
62               8  Barcelona             19  Barcelona        4   

    CODI_CONTAMINANT   ANY  MES  DIA   H01  ...   H20  V20   H21  V21   H22  \
58                10  2024    2    1  23.0  ...  47.0    V  44.0    V  38.0   
59                10  2024    2    2  24.0  ...  46.0    V  32.0    V  30.0   
60                10  2024    2    3  21.0  ...  23.0    V  24.0    V  29.0   
61                10  2024    2    4  30.0  ...  27.0    V  30.0    V  37.0   
62                10  2024    2    5  27.0  ...  33.0    V  26.0    V  30.0   

    V22   H23  V23   H24  V24  
58    V  21.0    V  22.0    V  
59    V  33.0    V  25.0    V  
60    V  29.0    V  

" ## Now merge the air_quality measurements and the location\n\nair_quality = air_quality_combined.merge(\n    right=air_station_locations,\n    left_on='ESTACIO',\n    right_on='Estacio',\n    how='left'\n).drop(columns=['Estacio'])\n\nprint(air_quality.head()) "

### Combining traffic files

In [14]:
## Create dataframe of measurements

traffic_files = [filename for url, filename in urls if "TRAMS" in filename]

# Covnert to pandas dataframes
traffic_dfs = [pd.read_csv(file) for file in traffic_files]
#print(traffic_dfs[0].head())

# Combine them into a single dataframe
traffic_combined = pd.concat(traffic_dfs, ignore_index=True)
print(traffic_combined.head())


## Create dataframe of station info
trams_info = pd.read_csv(trams_relacio_path)
# Only keep location data for each station
trams_locations = trams_info[['Tram', 'Coordenades']]

# Only keep one location point per section (tram=section)
def mean_coordinate(row):
    # print("entering function mean")
    # print(row)
    coord_text=row['Coordenades']
    numbers=coord_text.split(',')
    assert len(numbers) % 2 == 0
    lats = [float(x) for x in numbers[::2]]
    lons = [float(x) for x in numbers[1::2]]
    # print("infos")
    # print(lats)
    # print(type(lats[0]))
    lat = np.mean(lats)
    lon = np.mean(lons)
    return pd.Series(
        {
            'lat': lat,
            'lon': lon
        }
    )


trams_locations [['lat' , 'lon']] = trams_locations.apply(mean_coordinate, axis=1)
#print(trams_locations.head())


## Now merge the traffic measurements and the location
traffic = traffic_combined.merge(
    right=trams_locations,
    left_on='idTram',
    right_on='Tram',
    how='left'
)

traffic = traffic.drop(['Coordenades', 'Tram', 'estatPrevist'], axis=1)

print("RESULT FINA")
print(traffic.head())

   idTram            data  estatActual  estatPrevist
0       1  20240201102053            2             0
1       2  20240201102053            2             0
2       3  20240201102053            2             0
3       4  20240201102053            2             0
4       5  20240201102053            1             0
RESULT FINA
   idTram            data  estatActual       lat        lon
0       1  20240201102053            2  2.106769  41.382911
1       2  20240201102053            2  2.106769  41.383167
2       3  20240201102053            2  2.117372  41.385579
3       4  20240201102053            2  2.117281  41.385824
4       5  20240201102053            1  2.125109  41.387561


## Dataset issue

For some reason, the city does not provide information for the 'trams' with id greater than 527. In the `traffic` dataframe, there are some records with idTram between 535 and 539. They don't have a latitude/longitude. Let us drop them.

In [15]:
traffic = traffic.dropna(subset=['lat', 'lon'])

# Defining a traffic grid

Now, `traffic` contains the state (from 0-no car to 6-congestioned) of plenty of coordinates in Barcelona. But the density is not homogeneous, hence i will cut the city in a grid and compute an average value for traffic congestion.

In [16]:
# Getting the boundaries of the traffic information
minlat = traffic['lat'].min()
maxlat = traffic['lat'].max()
minlon = traffic['lon'].min()
maxlon = traffic['lon'].max()

#TODO add prints of the distance of the box here


nb_horizontal = 8
nb_vertical = 8

vertical_step = (maxlat - minlat) / nb_vertical
horizontal_step = (maxlon - minlon) / nb_horizontal


# Create new datafram from traffic

""" def getZone(row):
    lat = (row['lat'] - minlat) // vertical_step
    lon = (row['lon'] - minlon) // horizontal_step
    zone_number = (lat * nb_horizontal) + lon


traffic['zone'] = traffic.apply(getZone, axis=1) """
# BETTER WAY (vectorized)
# Create nb_vertical * nb_horizontal zones and assign every row a zone:
lat_idx = np.clip(np.floor((traffic['lat'] - minlat) / vertical_step).astype(int), 0, nb_vertical - 1)
lon_idx = np.clip(np.floor((traffic['lon'] - minlon) / horizontal_step).astype(int), 0, nb_horizontal - 1)
traffic['zone'] = lat_idx * nb_horizontal + lon_idx

assert traffic['zone'].max() <= nb_horizontal * nb_vertical

# Drop lat/lon as we will now use the zone
traffic.drop(['lat', 'lon'], axis=1)


print(traffic.head())
print(air_quality.head())

   idTram            data  estatActual       lat        lon  zone
0       1  20240201102053            2  2.106769  41.382911     2
1       2  20240201102053            2  2.106769  41.383167     2
2       3  20240201102053            2  2.117372  41.385579     3
3       4  20240201102053            2  2.117281  41.385824     3
4       5  20240201102053            1  2.125109  41.387561    11
    CODI_PROVINCIA  PROVINCIA  CODI_MUNICIPI   MUNICIPI  ESTACIO  \
58               8  Barcelona             19  Barcelona        4   
59               8  Barcelona             19  Barcelona        4   
60               8  Barcelona             19  Barcelona        4   
61               8  Barcelona             19  Barcelona        4   
62               8  Barcelona             19  Barcelona        4   

    CODI_CONTAMINANT   ANY  MES  DIA   H01  ...   H20  V20   H21  V21   H22  \
58                10  2024    2    1  23.0  ...  47.0    V  44.0    V  38.0   
59                10  2024    2    2 

# Merging all the data
We are then going to merge both of the dataframes


## Modification of the `traffic` dataframe
Traffic measurements are made more or less 10 minutes around o'clock hours. Then we will round the time to the nearest hour. Then we will melt the dataframe so that [hour, date] is a primary key.

In [17]:
# Covnert the 'data' (date) column of traffic dataframe
traffic["datetime"] = pd.to_datetime(traffic["data"], format="%Y%m%d%H%M%S")

# Traffic measurements are made more or less 10 minutes around o'clock hours. Then we will round the time to the nearest hour 
traffic["datetime_hour"] = traffic["datetime"].dt.round("h")
traffic["date"] = traffic["datetime_hour"].dt.date
traffic["hour"] = traffic["datetime_hour"].dt.hour


# Melting
traffic = traffic.pivot_table(
    index=['date', 'hour'],
    columns=['zone'],
    values='estatActual',
    aggfunc='first'
).reset_index()
print(traffic.head())



zone        date  hour  0  2  3  4  10  11  12  13  ...  47  51  52  53  54  \
0     2024-02-01    10  0  2  2  1   1   1   2   2  ...   1   2   0   1   0   
1     2024-02-01    11  0  2  2  1   1   2   2   2  ...   1   2   0   2   0   
2     2024-02-01    12  0  2  2  1   1   2   2   2  ...   1   2   0   2   0   
3     2024-02-01    13  0  2  2  1   2   2   2   2  ...   1   2   0   1   0   
4     2024-02-01    14  0  2  2  1   1   2   2   2  ...   1   2   0   2   0   

zone  55  60  61  62  63  
0      0   0   0   2   2  
1      0   0   0   2   2  
2      0   0   0   2   2  
3      0   0   0   2   2  
4      0   0   0   2   3  

[5 rows x 46 columns]


## Modification of the `air_quality` dataframe
The hours are currently in the columns, we are going to melt the dataframe to have a long one with dates in the rows.

We also want [hour,date] to be a primary key so that merging both dataframes does not explodes RAM usage.

In [18]:
# Get the hours columns name
hour_cols = [col for col in air_quality.columns if col.startswith("H") and len(col) == 3]

# Then use pandas melt method to reshape the DF

aq_melted = air_quality.melt(
    # Columns left untouched
    id_vars=["CODI_PROVINCIA", "PROVINCIA", "CODI_MUNICIPI", "MUNICIPI", "ESTACIO", "ANY", "MES", "DIA"], # IMPORTANT: i removed CODI_CONTAMINANT because i only kept PM10 temporarily

    #Columns to unpivot
    value_vars=hour_cols,

    #Name of the new column that will store the unpivoted columns names (H01 etc.)
    var_name="hour_str",

    #Name of the actual value column
    value_name="pollution_value"
)
    
# Clean the hour (H01->1)
aq_melted["hour"] = aq_melted["hour_str"].str.extract(r"H(\d{2})").astype("int32")



# Create the actual date column (and translate column names)
aq_melted["date"] = pd.to_datetime(
    aq_melted[["ANY", "MES", "DIA"]].rename(columns={"ANY": "year", "MES": "month", "DIA": "day"})
).dt.date

#Shift the H24 columns to hour 0 of the next day:
mask_24 = aq_melted["hour"] == 24
aq_melted.loc[mask_24, "hour"] = 0
aq_melted.loc[mask_24, "date"] = pd.to_datetime(aq_melted.loc[mask_24, "date"]) + pd.Timedelta(days=1)
aq_melted["date"] = pd.to_datetime(aq_melted["date"]).dt.date


# Pivot (CODI_CONTAMINANT,ESTACIO) so that [hour, date] is primary key
aq_pivoted = aq_melted.pivot_table(
    index=['date', 'hour'],
    columns=['ESTACIO'], # IMPORTANT: here i also remove CODI_CONTAMINANT since there is only one.
    values='pollution_value',
    aggfunc='first'
).reset_index()

# Simple flattening (for more meaningful column names)
""" aq_pivoted.columns = ['date', 'hour'] + [
    f'pollutant_{col[0]}_station_{col[1]}' 
    for col in aq_pivoted.columns[2:]
] """
aq_pivoted.columns = ['date', 'hour'] + [
    f'station_{col}' 
    for col in aq_pivoted.columns[2:]
]

print(aq_pivoted.head())


         date  hour  station_4  station_43  station_44  station_54  \
0  2024-02-01     1       23.0        30.0        40.0         NaN   
1  2024-02-01     2       19.0        45.0        35.0         NaN   
2  2024-02-01     3       19.0        45.0        34.0         NaN   
3  2024-02-01     4       19.0        41.0        32.0         NaN   
4  2024-02-01     5       19.0        36.0        29.0         NaN   

   station_57  station_58  
0        20.0        13.0  
1        20.0        12.0  
2        19.0        10.0  
3        24.0        15.0  
4        24.0        12.0  


In [19]:
# Now do the actual merging based on time
print("Dataframe `traffic`:")
print(traffic.head())
print("Dataframe aq: ")
print(aq_pivoted.head())
merged = pd.merge(
    traffic,
    aq_pivoted,
    how="left",
    left_on=["date", "hour"],
    right_on=["date", "hour"]
)

# is_primary_key = not traffic.duplicated(subset=['date', 'hour']).any()
# print(f"Is [date, hour] a primary key? {is_primary_key}")

print("Final DF")
print(merged.head(60))

print("taux de valeurs pourries")
print(merged.isna().sum().sum() / merged.size)

Dataframe `traffic`:
zone        date  hour  0  2  3  4  10  11  12  13  ...  47  51  52  53  54  \
0     2024-02-01    10  0  2  2  1   1   1   2   2  ...   1   2   0   1   0   
1     2024-02-01    11  0  2  2  1   1   2   2   2  ...   1   2   0   2   0   
2     2024-02-01    12  0  2  2  1   1   2   2   2  ...   1   2   0   2   0   
3     2024-02-01    13  0  2  2  1   2   2   2   2  ...   1   2   0   1   0   
4     2024-02-01    14  0  2  2  1   1   2   2   2  ...   1   2   0   2   0   

zone  55  60  61  62  63  
0      0   0   0   2   2  
1      0   0   0   2   2  
2      0   0   0   2   2  
3      0   0   0   2   2  
4      0   0   0   2   3  

[5 rows x 46 columns]
Dataframe aq: 
         date  hour  station_4  station_43  station_44  station_54  \
0  2024-02-01     1       23.0        30.0        40.0         NaN   
1  2024-02-01     2       19.0        45.0        35.0         NaN   
2  2024-02-01     3       19.0        45.0        34.0         NaN   
3  2024-02-01     4     

# TODO

- nettoyer air quality: enlever les colonnes de polluants qui ne sont pas dans la référence(https://opendata-ajuntament.barcelona.cat/data/dataset/6960936a-95ed-4cc4-a6ec-e089197ccd8b/resource/c122329d-d26d-469e-bf9e-8efa10e4c127/download/qualitat_aire_contaminants.csv)
dans la référence, les polluants avec une * sont plus précis.


- se concentrer sur un seul/2 polluant (PM10=10, PM2.5=9)

- Gérer les données vides



We will focus first only on PM10 pollutant, 